In [199]:
import numpy as np
import pandas as pd

from models import Mw, Firm
#from funcs import snap
from scipy.interpolate import interp1d #functional interpolation


ImportError: cannot import name 'inverse_mw' from 'funcs' (/Users/Aex/Desktop/Research/NoRegretLearning/funcs.py)

In [29]:
### players
# price setters
# consumers - play no regret learning

In [196]:
#parameters
N = 2
Beta = 0.9
Nu = 0.1

#models
firms =  [Firm(i,Beta) for i in range(N)] 
mw = Mw(N,Nu)

In [45]:
#simulate some simple play
for i in range(24):
    prices = [1,2]
    for j in range(len(firms)):
        firms[j].updateShares(mw.weights[j]) #update shares
        firms[j].setPrice(prices[j]) #update prices
    mw.update(np.array(prices))

firms[0].dpv()

6.96401530498293

In [395]:
### code up policy iteration


#setup
n_points = 10
s_list = [ (1/(n_points))*(i+1) for i in range(0,n_points)]  #values for weights
p_list = [ (1/(n_points))*(i+1) for i in range(0,n_points)] #values for prices - allows the boundaries
v0 = [1]*len(s_list)

#hardcoded for now, but will make functions later
policyB = 0.1 ## policy for player 2 - make deterministic for now
Beta = 0.9
Nu = 0.2

In [399]:
# lets try it like a function
v_curr = v0
p_curr = [1]*len(s_list) #policy function ('grid')



### idea: start with a grid of states. generate
# i) an interpolated value function (for the player to estimte future values)
# ii) an interpolated policy function (to simulate the other players play)


for i in range(100):
    #loop through states, and for each, pick an optimal policy, and update corresponding value
    v_new = [-1]*len(v_curr) #prep new list
    # define functions for analysis
    interp_v_func = interp1d(s_list, #grid points
                             v_curr, #values
                              kind='linear', fill_value='extrapolate')
    interp_p_func = interp1d(s_list, #grid points
                             p_curr, #price function
                             kind='linear', fill_value='extrapolate') #we use thids for other player!
    for j in range(len(s_list)):
        s = s_list[j]
        values = [-1]*len(p_list)
        for i in range(len(p_list)):
            p = p_list[i]
            flow_u = p*s
            new_weight_a = s*(1 - (Nu*p))
            #new_weight_b = (1-s)*(1 - (Nu*policyB))
            new_weight_b = (1-s)*(1 - (Nu*interp_p_func(1-s)))
            new_state = new_weight_a / (new_weight_a + new_weight_b)
            #future_u = Beta*v_curr[snap(new_state,s_list)]
            future_u = Beta*interp_v_func(new_state)
            ### interpolating function
            values[i]= flow_u + future_u
        v_new[j] = np.array(values).max()
        p_curr[j] = p_list[np.array(values).argmax()]
    v_curr = v_new





In [400]:
s_list

[0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9,
 1.0]

In [401]:
#policy as a function of states
p_curr

[0.1, 0.1, 0.1, 0.4, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [402]:
v_curr

[1.7076232208420947,
 2.6096278186089488,
 3.3868414288230553,
 4.161754219520731,
 4.999893754404451,
 5.246405382311428,
 5.598256667501147,
 6.1839534599898185,
 7.1960093303109,
 9.999760947410016]